In [1]:
import os, shutil

In [2]:
base_dir = './images_base'


train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


train_cats_dir = os.path.join(train_dir, 'cats')

train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')

validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')

test_dogs_dir = os.path.join(test_dir, 'dogs')

In [3]:
from keras.applications import VGG16
from keras import models,layers,optimizers

conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(150, 150, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable=False

model.summary()

Using TensorFlow backend.


58892288/58889256 [==============================] - 354s 6us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 15,763,521
Trainable params: 1,048,833
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
sgd = optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='dogsVScats.model.weights_vgg16_use.hdf5', verbose=1, 
                               save_best_only=True)
history = model.fit_generator(
          train_generator,
          steps_per_epoch=100,
          epochs=5,
          validation_data=validation_generator,
          validation_steps=50, 
          callbacks=[checkpointer], 
          verbose=1)

Epoch 1/5
100/100 [==============================] - 222s 2s/step - loss: 0.4226 - accuracy: 0.8145 - val_loss: 0.2278 - val_accuracy: 0.9010

Epoch 00001: val_loss improved from inf to 0.22779, saving model to dogsVScats.model.weights_vgg16_use.hdf5
Epoch 2/5
100/100 [==============================] - 224s 2s/step - loss: 0.2118 - accuracy: 0.9155 - val_loss: 0.4586 - val_accuracy: 0.8530

Epoch 00002: val_loss did not improve from 0.22779
Epoch 3/5
100/100 [==============================] - 224s 2s/step - loss: 0.1484 - accuracy: 0.9460 - val_loss: 0.5805 - val_accuracy: 0.8540

Epoch 00003: val_loss did not improve from 0.22779
Epoch 4/5
100/100 [==============================] - 224s 2s/step - loss: 0.0951 - accuracy: 0.9690 - val_loss: 0.4596 - val_accuracy: 0.8550

Epoch 00004: val_loss did not improve from 0.22779
Epoch 5/5
100/100 [==============================] - 224s 2s/step - loss: 0.0784 - accuracy: 0.9725 - val_loss: 0.3753 - val_accuracy: 0.8980

Epoch 00005: val_loss di